In [1]:
import numpy as np
import xarray as xr
import netCDF4
import os
import datetime
import matplotlib.pyplot as plt 
from matplotlib import cm
import shutil
from datetime import datetime, timedelta
import glob
import datetime as dt
from os import path
import fsspec
import git 
import json
import sys
import yaml

from functions_verification import *

## Time Range Definition

In [2]:
today= datetime.today()

In [3]:
today_string=datetime.today().strftime('%Y%m%d')
today_string2=datetime.today().strftime('%Y-%m-%d')

In [4]:
startyear=2018
endyear=today.year

## Data Read In

### Folder Structure

In [5]:
with open(r"/Volumes/lexplore_hd/scripts/verification_2024.yml", "r") as f:
    directories = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
#log("Creating directories")
#for d in directories.values():
#    if not os.path.exists(d):
#        os.makedirs(d)

In [7]:
# defining folders
output_folder=directories["plot_output_folder"]

#tchain_folder = directories["tchain_folder"]
tchain_folder_netcdf=directories["tchain_data_folder"]

ctd_folder = directories["ctd_folder"]
ctd_folder_netcdf = ctd_folder+"data/Level2B/"

pp_mooring_folder=directories["pp_mooring_folder"]
pp_mooring_par_folder_netcdf=pp_mooring_folder+"data/Level2/LexplorePPMooringPAR/"
pp_mooring_do_folder_netcdf=pp_mooring_folder+"data/Level2/LexplorePPMooringDO/"

#thetis_folder = directories["thetis_folder"]
thetis_folder_netdf = directories["thetis_data_folder"]

#idronaut_folder = directories["idronaut_folder"]
idronaut_folder_netcdf = directories["idronaut_data_folder"]

scripts_folder=directories["scripts_folder"]

In [8]:
# get faulty dates for idronaut
f = open(scripts_folder+"idronaut_faulty.json")
dates2exclude = json.load(f)

In [9]:
log("pull data")
try:
    #g = git.cmd.Git(thetis_folder)
    #g.pull()
    g = git.cmd.Git(pp_mooring_folder)
    g.pull()
    #g = git.cmd.Git(tchain_folder)
    #g.pull()
    #g = git.cmd.Git(idronaut_folder)
    #g.pull()
    g = git.cmd.Git(ctd_folder)
    g.pull()
except:
    log("Failed to pull data")

19:11:40.496519   pull data


Pull Thetis

In [12]:
os.chdir(thetis_folder_netdf)

In [13]:
!pwd

/Volumes/lexplore_hd/core_data/thetis-multi-instrument-profiler/data/Level2


In [14]:
!python /Volumes/lexplore_hd/core_data/thetis-multi-instrument-profiler/scripts/download_remote_data.py -w
!rm /Volumes/lexplore_hd/core_data/thetis-multi-instrument-profiler/data/Level2/L2_THETIS_GRID_20191202_20191212.nc



Attempting to sync s3://eawag-data/gitlab.renkulab.io/lexplore/thetis-multi-instrument-profiler/data with /Volumes/lexplore_hd/core_data/thetis-multi-instrument-profiler/data
download: s3://eawag-data/gitlab.renkulab.io/lexplore/thetis-multi-instrument-profiler/data/Level2/L2_THETIS_GRID_20191202_20191212.nc to ./L2_THETIS_GRID_20191202_20191212.nc

Download complete.


Pull Idronaut

In [15]:
os.chdir(idronaut_folder_netcdf)

In [16]:
!python /Volumes/lexplore_hd/core_data/idronaut-automatic-profiler/scripts/download_remote_data.py -w

Attempting to sync s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data with /Volumes/lexplore_hd/core_data/idronaut-automatic-profiler/data
download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level0/Deployment2/2024-01-17@22.21.54.txt to ../Level0/Deployment2/2024-01-17@22.21.54.txt

download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level0/Deployment2/2024-01-18@02.21.53.txt to ../Level0/Deployment2/2024-01-18@02.21.53.txt

download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level0/Deployment2/2024-01-18@06.21.55.txt to ../Level0/Deployment2/2024-01-18@06.21.55.txt

download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level0/Deployment2/2024-01-18@00.21.58.txt to ../Level0/Deployment2/2024-01-18@00.21.58.txt

download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level0/Deployment2/2024-01-1

download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level1/L1_LexploreIdronaut_D2_20240118_222153.nc to ../Level1/L1_LexploreIdronaut_D2_20240118_222153.nc

download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level1/L1_LexploreIdronaut_D2_20240118_182203.nc to ../Level1/L1_LexploreIdronaut_D2_20240118_182203.nc

download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level1/L1_LexploreIdronaut_D2_20240119_022205.nc to ../Level1/L1_LexploreIdronaut_D2_20240119_022205.nc

download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level1/L1_LexploreIdronaut_D2_20240118_202142.nc to ../Level1/L1_LexploreIdronaut_D2_20240118_202142.nc

download: s3://eawag-data/gitlab.renkulab.io/lexplore/idronaut-automatic-profiler/data/Level1/L1_LexploreIdronaut_D2_20240119_002205.nc to ../Level1/L1_LexploreIdronaut_D2_20240119_002205.nc

download: s3://eawag-data/gitlab.renkula

Pull Temperature Chain

In [17]:
os.chdir(tchain_folder_netcdf)

In [18]:
!python /Volumes/lexplore_hd/core_data/thermister-chain/scripts/download_remote_data.py -w



Attempting to sync s3://eawag-data/gitlab.renkulab.io/lexplore/thermister-chain/data with /Volumes/lexplore_hd/core_data/thermister-chain/data
download: s3://eawag-data/gitlab.renkulab.io/lexplore/thermister-chain/data/Level0/v2/LeXPLORE_EAST_TempChain_2024-01-18.dat to ../Level0/v2/LeXPLORE_EAST_TempChain_2024-01-18.dat

download: s3://eawag-data/gitlab.renkulab.io/lexplore/thermister-chain/data/Level0/v2/LeXPLORE_EAST_TempChain_2024-01-21.dat to ../Level0/v2/LeXPLORE_EAST_TempChain_2024-01-21.dat

download: s3://eawag-data/gitlab.renkulab.io/lexplore/thermister-chain/data/Level0/v2/LeXPLORE_EAST_TempChain_2024-01-19.dat to ../Level0/v2/LeXPLORE_EAST_TempChain_2024-01-19.dat

download: s3://eawag-data/gitlab.renkulab.io/lexplore/thermister-chain/data/Level0/v2/LeXPLORE_EAST_TempChain_2024-01-20.dat to ../Level0/v2/LeXPLORE_EAST_TempChain_2024-01-20.dat

download: s3://eawag-data/gitlab.renkulab.io/lexplore/thermister-chain/data/Level2/L2_LexploreTemperatureChain_v2_20240101_000000.nc 

### read netcdf files

In [ ]:

thetis_all=read_in_allfiles(folder=thetis_folder_netdf)
pp_do_all=read_in_allfiles(folder=pp_mooring_do_folder_netcdf)
pp_par_all=read_in_allfiles(folder=pp_mooring_par_folder_netcdf)
tchain_all=read_in_allfiles(folder=tchain_folder_netcdf)
#tchain_all=tchain_all.temp.where(tchain_all.interp_temp<1)
tchain_all=tchain_all.temp.where(tchain_all.temp_qual==0)
tchain_all.attrs["source"]="Temperature Chain"
idronaut_all=read_in_allfiles(folder=idronaut_folder_netcdf)
ctd_all=read_in_allfiles(folder=ctd_folder_netcdf)


In [ ]:
thetis_depths_limit=tchain_all.depth.values[3:39]*-1

In [ ]:
manual_depths=[ -1.  ,  -1.5 ,  -2.  ,  -2.5 ,  -3.  ,  -4.  ,  -5.  ,
        -6.  ,  -7.  ,  -8.  ,  -9.  , -10.  ,
       -15.  , -21.  , -24.  ,
       -30.  , -36.  , -39.  , -45.  , -51.  ]

In [ ]:
# create plotting dictionaries
idronaut_thetis_dict=["chla","temp","sal","cond"]
idronaut_ctd_dict=["chla","temp","pH"]
thetis_ctd_dict=["chla","temp","sal","cond","do"]
thetis_dict=["chla","temp","sal","cond","do","par"]
idronaut_dict=["chla","temp","sal","cond","pH"]
ctd_dict=["chla","temp","sal","cond","pH","do"]
cc_dict=["depth","date"]

In [ ]:
thetis_all.coords['depth'] = (thetis_all.coords['depth'] * -1)
idronaut_all.coords['Press'] = (idronaut_all.coords['Press'] * -1)
tchain_all.coords['depth'] = (tchain_all.coords['depth'] * -1)
ctd_all.coords['depth'] = (ctd_all.coords['depth'] * -1)
idronaut_all=idronaut_all.rename({'Press': 'depth'})

In [ ]:
idronaut_all=idronaut_all.rename({'Chl': 'chla','Sal': 'sal','Temp': 'temp','Cond': 'cond'})
ctd_all=ctd_all.rename({'Chl_A': 'chla','SALIN': 'sal','Temp': 'temp','Cond': 'cond','DO_mg': 'do'})

In [ ]:
# reducing depth information
# reduce to same depths
log("reducing depth information")
thetis_all_less_depths=thetis_all[thetis_dict].sel(depth=manual_depths, method='nearest')
ctd_all_less_depths=ctd_all[ctd_dict].sel(depth=manual_depths, method='nearest')
tchain_all_less_depths=tchain_all.sel(depth=manual_depths, method='nearest')
idronaut_all_less_depths=idronaut_all[idronaut_dict].sel(depth=manual_depths, method='nearest')

In [ ]:
thetis_all_less_depths=thetis_all.sel(depth=thetis_depths_limit, method='nearest')
ctd_all_less_depths=ctd_all.sel(depth=thetis_depths_limit, method='nearest')
tchain_all_less_depths=tchain_all.sel(depth=thetis_depths_limit, method='nearest')
idronaut_all_less_depths=idronaut_all.sel(depth=thetis_depths_limit, method='nearest')

In [ ]:
thetis_all_less_depths_night=get_night(thetis_all_less_depths)
idronaut_all_less_depths_night=get_night(idronaut_all_less_depths)
ctd_all_less_depths_night=get_night(ctd_all_less_depths)

In [ ]:
pp_par_all_30mins=pp_par_all.resample(time='30min').mean()
tchain_all_30mins=tchain_all_less_depths.resample(time='30min').mean()  
pp_do_all_30mins=pp_do_all.resample(time='30min').mean()
pp_do_all_30mins.attrs["source"]="PP Mooring"
pp_do_all_30mins.attrs["units"]=thetis_all.do.units
pp_par_all_30mins.attrs["source"]="PP Mooring"
pp_par_all_30mins.attrs["units"]=thetis_all.par.units
tchain_all_30mins.attrs["source"]="Temperature Chain"
tchain_all_30mins.attrs["units"]=thetis_all.temp.units

In [ ]:
plot_clim(thetis_all,"temp",output_folder=output_folder,cmap="inferno",res="daily")
plot_clim(thetis_all,"temp",output_folder=output_folder,cmap="inferno",res="monthly")
plot_samples(thetis_all,"temp",output_folder=output_folder,cmap="viridis",res="monthly")
plot_samples(idronaut_all,"Temp",output_folder=output_folder,cmap="viridis",res="monthly")
plot_samples(tchain_all,"tchain",output_folder=output_folder,cmap="viridis",res="monthly")
plot_samples(ctd_all,"Temp",output_folder=output_folder,cmap="viridis",res="monthly")

In [ ]:
thetis_tchain_intersect_nearest,tchain_thetis_intersect_nearest=verification(thetis_all_less_depths,tchain_all_30mins)
idronaut_tchain_intersect_nearest,tchain_idronaut_intersect_nearest=verification(idronaut_all_less_depths,tchain_all_30mins)
thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest=verification(thetis_all_less_depths,ctd_all_less_depths)
tchain_ctd_intersect_nearest,ctd_tchain_intersect_nearest=verification(tchain_all_30mins,ctd_all_less_depths)
thetis_idronaut_intersect_nearest,idronaut_thetis_intersect_nearest=verification(thetis_all_less_depths,idronaut_all_less_depths)
ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest=verification(ctd_all_less_depths,idronaut_all_less_depths)
idronaut_tchain_intersect_nearest,tchain_idronaut_intersect_nearest=verification(idronaut_all_less_depths,tchain_all_30mins)
thetis_idronaut_intersect_nearest_night,idronaut_thetis_intersect_nearest_night=verification(thetis_all_less_depths_night,idronaut_all_less_depths_night)
thetis_pp_par_intersect_nearest,pp_par_thetis_intersect_nearest=verification(thetis_all_less_depths,pp_par_all_30mins)
ctd_pp_do_intersect_nearest,pp_do_ctd_intersect_nearest=verification(ctd_all_less_depths,pp_do_all_30mins) 
thetis_idronaut_intersect_nearest_night,idronaut_thetis_intersect_nearest_night=verification(thetis_all_less_depths_night,idronaut_all_less_depths_night)
ctd_idronaut_intersect_nearest_night,idronaut_ctd_intersect_nearest_night=verification(ctd_all_less_depths_night,idronaut_all_less_depths_night)
thetis_pp_do_intersect_nearest,pp_do_thetis_intersect_nearest=verification(thetis_all_less_depths,pp_do_all_30mins)



In [ ]:
pp_do_thetis_intersect_nearest=pp_do_thetis_intersect_nearest.load()
pp_par_thetis_intersect_nearest=pp_par_thetis_intersect_nearest.load()
tchain_thetis_intersect_nearest=tchain_thetis_intersect_nearest.load()
tchain_idronaut_intersect_nearest=tchain_idronaut_intersect_nearest.load()
pp_do_ctd_intersect_nearest=pp_do_ctd_intersect_nearest.load()

In [ ]:
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"temp","Temp",output_folder,cc="date")
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"chla","Chl_A",output_folder,cc="date")
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"do","DO_mg",output_folder,cc="date")
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"cond","Cond",output_folder,cc="date")
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"sal","SALIN",output_folder,cc="date")
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"temp","Temp",output_folder,cc="depth")
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"chla","Chl_A",output_folder,cc="depth")
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"do","DO_mg",output_folder,cc="depth")
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"cond","Cond",output_folder,cc="depth")
ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,"sal","SALIN",output_folder,cc="depth")



In [ ]:
ver_plotting(thetis_idronaut_intersect_nearest,idronaut_thetis_intersect_nearest,"temp","Temp",output_folder,cc="date")
ver_plotting(thetis_idronaut_intersect_nearest,idronaut_thetis_intersect_nearest,"temp","Temp",output_folder,cc="depth")



In [ ]:
ver_plotting(ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest,"Temp","Temp",output_folder,cc="date")
ver_plotting(ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest,"Temp","Temp",output_folder,cc="depth")
ver_plotting(ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest,"Chl_A","Chl",output_folder,cc="date")
ver_plotting(ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest,"Chl_A","Chl",output_folder,cc="depth")
ver_plotting(ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest,"pH","pH",output_folder,cc="date")
ver_plotting(ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest,"pH","pH",output_folder,cc="depth")



In [ ]:
ver_plotting(ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest,"Temp","Temp",output_folder,cc="hour",grid=False)



In [ ]:
dataset1_time=dataset1.time.values
dataset2_time=dataset2.time.values
#rounding to the closest hour
dataset1_hour=np.array(dataset1_time, dtype='datetime64[h]')
dataset2_hour=np.array(dataset2_time, dtype='datetime64[h]')
# precise overlap over the next hour
time_intersect=np.intersect1d(dataset1_hour,dataset2_hour)
# select common overlap time steps
dataset1_dataset2_intersect_nearest=dataset1.sel(time=time_intersect, method='nearest')
dataset2_dataset1_intersect_nearest=dataset2.sel(time=time_intersect, method='nearest')

In [ ]:
for i in thetis_ctd_dict:
    for b in cc_dict:

In [ ]:
def ver_plotting(dataset1,dataset2,var1,var2,output_folder,cc="depth",grid=True):
    today_string=datetime.today().strftime('%Y%m%d')
    z=dataset1.depth.values
    z_scaled=(z-z.min())/z.ptp()
    colors=plt.cm.viridis(z_scaled,len(z))
    day_in_year,date_index,hour=get_time(dataset1)
    day_scaled=(np.array(day_in_year)-min(day_in_year))/np.ptp(day_in_year)
    colors_day=plt.cm.viridis(day_scaled)
    date_scaled=(np.array(date_index)-min(date_index))/np.ptp(date_index)
    colors_date=plt.cm.viridis(date_scaled)
    hour_scaled=(np.array(hour)-min(hour))/np.ptp(hour)
    colors_hour=plt.cm.viridis(hour_scaled)
    
    if cc=="date":
        amount_dates=len(date_index)
        fig = plt.figure(figsize=(13, 7))
        for i in range(amount_dates):
            plt.scatter(dataset1[var1][:,i],dataset2[var2][:,i],color=colors_date[i],alpha=0.5, label=date_index[i])
        title=dataset1.source+" vs "+dataset2.source+" "+ dataset1[var1].long_name
        plt.title(title)
        color_code=cc
        cbar=plt.colorbar(ticks=[0, 1], label=color_code)
    if cc=="depth":
        amount_depth=len(z)
        fig = plt.figure(figsize=(13, 7))
        for i in range(amount_depth):
            plt.scatter(dataset1[var1][i,:],dataset2[var2][i,:],color=colors[i],alpha=0.5, label=z[i])
        title=dataset1.source+" vs "+dataset2.source+" "+ dataset1[var1].long_name
        plt.title(title)
        color_code=cc
        cbar=plt.colorbar(ticks=[0, 1], label=color_code)
    if cc=="hour":
        amount_depth=len(hour)
        fig = plt.figure(figsize=(13, 7))
        for i in range(amount_depth):
            plt.scatter(dataset1[var1][:,i],dataset2[var2][:,i],color=colors_hour[i],alpha=0.5, label=hour[i])
        title=dataset1.source+" vs "+dataset2.source+" "+ dataset1[var1].long_name
        plt.title(title)
        color_code=cc
        cbar=plt.colorbar(ticks=[0, 1], label=color_code)
        
    plt.xlabel(dataset1.source+" "+dataset1[var1].units)
    plt.ylabel(dataset2.source+" "+dataset2[var2].units)
    #plt.legend()
    if cc=="date":
        cbar.ax.set_yticklabels([min(date_index),max(date_index)]) 
    if cc=="depth":
        cbar.ax.set_yticklabels([min(z),max(z)])
    if cc=="hour":
        cbar.ax.set_yticklabels([min(hour),max(hour)]) 
    if grid==True:
        plt.grid()
    xpoints=ypoints=plt.xlim()
    plt.plot(xpoints,ypoints,color="k",lw=3,scalex=False,scaley=False,alpha=0.2)
    fig.savefig(os.path.join(output_folder,today_string+" "+title+" "+color_code+".pdf"))
    plt.close('all')
    return print("Figure saved: "+title+" "+color_code+".pdf")

In [ ]:
# experiment

thetis_all.sel(time=slice("2021-02-26","2022-10-26"))